In [1]:
!pip install torchsummaryX adamp librosa --quiet

In [2]:
import librosa
import numpy as np
from torchsummaryX import summary
import sklearn
import sklearn.metrics
import gc
import zipfile
import pandas as pd
import random
from tqdm.autonotebook import tqdm
import os
import datetime
import torch.nn as nn
import torch
from adamp import AdamP
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print("Device: ", device)

Device:  cpu


In [3]:
from google.colab import drive # Link your drive if you are a colab user
drive.mount('/content/drive') 

Mounted at /content/drive


In [4]:
root = '/content/drive/MyDrive/11685/Project/FluteDetection/wav_resampled16kHz'

# Save MFCC files for train, val, test dataset

In [ ]:
# def mfcc_generation(partition, win_length):
#   wav_dir = root + '/' + partition + '/wav/'
#   mfcc_dir = root + '/' + partition + '/mfcc_'+ str(win_length)+'/'
#   wav_files = sorted(os.listdir(wav_dir))
#   for file_name in wav_files:
#     file_path = wav_dir + file_name
#     signal, rate = librosa.load(file_path, sr = None)
#     # sampling rate 16KHz, hop length in seconds: 0.01s, window length in seconds: 0.01s
#     mfcc = librosa.feature.mfcc(y=signal, sr=rate, hop_length=160, window= 'hann',n_mfcc = 20, win_length=win_length, n_fft = 1024, n_mels = 20)
#     mfcc = mfcc.T
#     new_file_name = file_name[:-4] + '.npy'
#     with open(mfcc_dir + new_file_name, 'wb') as f:
#       np.save(f, mfcc)

In [ ]:
# mfcc_generation('train',320)
# mfcc_generation('train', 480)
# mfcc_generation('dev', 320)
# mfcc_generation('dev', 480)
# mfcc_generation('test', 320)
# mfcc_generation('test', 480)

# Generate labels for note identity

In [5]:
Notes = ['NOTE', 'newnote', 'new note', '0.8']
Ornaments = ['crann_nt', 'crann_ct', 'ct', 'cr', 'ct_pShrt', 'shake', 'lngRoll_nt', 'shRoll_str', 'shRoll_ct', 'str', 'lngRoll_str', 'lngRoll_ct', 'lngRoll_nt_sl', 'shRoll_nt']

In [6]:
def embedding(identity):
  result = '0'
  if (identity in Notes):
    result = '1'
  elif (identity in Ornaments):
    result = '2'
  return result

In [17]:
def labels_generation(partition):
  annotations_dir = root + '/' + partition + '/annotations/'
  labels_dir = root + '/' + partition + '/note_identity/'
  mfcc_dir = root + '/' + partition + '/mfcc/'
  annotation_files = sorted(os.listdir(annotations_dir))
  count_1_total = 0
  count_2_total = 0
  count_3_total = 0
  for file_name in annotation_files:
    identities = []
    onsets = []
    offsets = []
    annotation_file_path  = annotations_dir + file_name
    mfcc_file_path = mfcc_dir + file_name[:-4] + '.npy'
    length = np.load(mfcc_file_path).shape[0]
    with open(annotation_file_path, 'rb') as f:
      lines = f.readlines()
      for information in lines:
        information_split = str(information)[2:-5].split("\\t")
        onset = information_split[1:2]
        offset = information_split[2:3]
        identity = information_split[4:5][0]
        identity = embedding(identity)
        identities += identity
        onsets += onset
        offsets += offset
    f.close()
    onsets_final = [round(100*float(tt)) for tt in onsets]
    offsets_final = [round(100*float(tt)) for tt in offsets]
    labels = np.zeros((length, 1))
    for index in range(len(onsets_final)):
      if offsets_final[index] < length:
        labels[onsets_final[index]:offsets_final[index]] = int(identities[index])
    count_1 = 0
    count_2 = 0
    count_3 = 0
    unique, counts = np.unique(labels, return_counts=True)
    
    if len(counts) == 3:
      count_1 = counts[0]
      count_2 = counts[1]
      count_3 = counts[2]
    elif len(counts) == 2:
      count_1 = counts[0]
      count_2 = counts[1]
    count_1_total += count_1
    count_2_total += count_2
    count_3_total += count_3
    print(dict(zip(unique, counts)))
  print(count_1_total, count_2_total, count_3_total)
    # print(dict(zip(unique, counts)))
#     with open(labels_dir + file_name[:-4] + '.npy', 'wb') as f:
#       np.save(f, labels)

In [18]:
labels_generation('train')
labels_generation('dev')
labels_generation('test')

{0.0: 261, 1.0: 2518, 2.0: 647}
{0.0: 276, 1.0: 2594, 2.0: 507}
{0.0: 149, 1.0: 1396, 2.0: 153}
{0.0: 70, 1.0: 1625}
{0.0: 106, 1.0: 1401, 2.0: 184}
{0.0: 341, 1.0: 2355, 2.0: 257}
{0.0: 1077, 1.0: 3221, 2.0: 147}
{0.0: 865, 1.0: 3110, 2.0: 188}
{0.0: 1055, 1.0: 2711, 2.0: 385}
{0.0: 1347, 1.0: 2696, 2.0: 386}
{0.0: 535, 1.0: 2333, 2.0: 621}
{0.0: 314, 1.0: 1245, 2.0: 152}
{0.0: 395, 1.0: 1770, 2.0: 1054}
{0.0: 451, 1.0: 2054, 2.0: 503}
{0.0: 257, 1.0: 1602, 2.0: 109}
{0.0: 257, 1.0: 1326, 2.0: 246}
{0.0: 261, 1.0: 1237, 2.0: 320}
{0.0: 326, 1.0: 2340, 2.0: 269}
{0.0: 249, 1.0: 2363, 2.0: 91}
{0.0: 466, 1.0: 2451, 2.0: 89}
{0.0: 387, 1.0: 2438, 2.0: 88}
{0.0: 407, 1.0: 2457, 2.0: 67}
{0.0: 287, 1.0: 2404, 2.0: 190}
{0.0: 239, 1.0: 2430, 2.0: 199}
{0.0: 471, 1.0: 2454, 2.0: 178}
{0.0: 205, 1.0: 1416, 2.0: 150}
{0.0: 155, 1.0: 1471, 2.0: 128}
{0.0: 142, 1.0: 1488, 2.0: 124}
{0.0: 265, 1.0: 2871, 2.0: 1121}
{0.0: 606, 1.0: 2975, 2.0: 970}
{0.0: 464, 1.0: 2581, 2.0: 189}
{0.0: 247, 1.0: 27

# Test Labels Generation

In [ ]:
def test_labels_generate(path):
    transcript_names = sorted(os.listdir(path))
    transcripts = []
    for name in transcript_names:
      transcript = np.load(path + name)
      transcripts.append(transcript)
    transcripts_final = np.concatenate(transcripts, axis=0)
    return transcripts_final

# Train/Validation Dataset

In [ ]:
# Dataset class to load train and validation data

from calendar import day_abbr
from sklearn.preprocessing import normalize


class AudioDataset(torch.utils.data.Dataset):

    def __init__(self, data_path, context, offset=0, partition= "train", limit=-1, shuffle=True, Normalization=True, mfcc_list=["mfcc", "mfcc_320", "mfcc_480"]): # Feel free to add more arguments

        self.context = context
        self.offset = context
        self.data_path = data_path
        self.normalization = Normalization
        self.mfcc_list = mfcc_list
        
        self.mfcc_list = []
        self.transcripts = []
        for index, mfcc_file in enumerate(mfcc_list):
            if partition == "train":
                self.mfcc_dir = data_path + '/' + partition + "/" + mfcc_file + "/"
                self.transcript_dir = data_path + '/' + partition + "/note_identity/"
            else:
                self.mfcc_dir = data_path + '/' + partition + "/" + mfcc_file + "/"
                self.transcript_dir = data_path + '/' + partition + "/note_identity/"

            mfcc_names = sorted(os.listdir(self.mfcc_dir))
            transcript_names = sorted(os.listdir(self.transcript_dir))
            assert len(mfcc_names) == len(transcript_names)

            self.mfccs = []

            for i in range(0, len(mfcc_names)):
    
                mfcc = np.load(self.mfcc_dir + mfcc_names[i])
                self.mfccs.append(mfcc)
            #   Optionally do Cepstral Normalization of mfcc
            #   Load the corresponding transcript
                if index == 0:
                    transcript = np.load(self.transcript_dir + transcript_names[i])
                    self.transcripts.append(transcript)

            # if partition == "train":
            #     if shuffle == True:
            #         Pairs = list(zip(self.mfccs, self.transcripts))
            #         random.shuffle(Pairs)
            #         self.mfccs, self.transcripts = zip(*Pairs)

            # Each mfcc is of shape T1 x 20, T2 x 20, ...
            # Each transcript is of shape (T1+2) x 20, (T2+2) x 20

            # TODO: Concatenate all mfccs in self.X such that the final shape is T x 20 (Where T = T1 + T2 + ...) 
            self.mfccs = np.concatenate(self.mfccs, axis=0)
            self.length = len(self.mfccs)
            # TODO: Concatenate all transcripts in self.Y such that the final shape is (T,) meaning, each time step has one phoneme output
            if index == 0:
                self.transcripts = np.concatenate(self.transcripts, axis=0)
            # Hint: Use numpy to concatenate
            # Take some time to think about what we have done. self.mfcc is an array of the format (Frames x Features). Our goal is to recognize phonemes of each frame
            # From hw0, you will be knowing what context is. We can introduce context by padding zeros on top and bottom of self.mfcc
            if context != 0:
                zero_paddings = np.zeros((context, 20))
                up_paded = np.vstack((zero_paddings, self.mfccs))
                down_paded = np.vstack((up_paded, zero_paddings))
                self.mfccs = down_paded
                
            self.mfcc_list.append(self.mfccs)
        self.mfccs = np.stack(self.mfcc_list)


    def __len__(self):
        return self.length

    def __getitem__(self, ind):
        
        start_index = ind + self.offset - self.context
        
        ## Calculate ending timestep using offset and context (1 line)
        end_index = ind + self.offset + self.context + 1

        frames = self.mfccs[:, start_index:end_index, :]
        
        # TODO: Based on context and offset, return a frame at given index with context frames to the left, and right.
        # After slicing, you get an array of shape 2*context+1 x 15. But our MLP needs 1d data and not 2d.
        height = frames.shape[0]
        width = frames.shape[1]
        if self.normalization == True:
          frames = frames - frames.mean(axis=0, keepdims=True)
          # frames_variance = np.var(frames, axis=0)
          # frames = np.divide(frames, np.tile(frames_variance, (height, 1)))
        frames = torch.FloatTensor(frames) # Convert to tensors
        onset = torch.tensor(self.transcripts[ind])       
        
        return frames, onset

# Test Dataset

In [ ]:
class AudioTestDataset(torch.utils.data.Dataset):
    # TODO: Create a test dataset class similar to the previous class but you dont have transcripts for this
    # Imp: Read the mfccs in sorted order, do NOT shuffle the data here or in your dataloader.
    def __init__(self, data_path, context, offset=0, limit=-1, Normalization=True, mfcc_list=["mfcc", "mfcc_320", "mfcc_480"]): # Feel free to add more arguments

        self.context = context
        self.offset = context
        self.data_path = data_path
        self.normalization = Normalization
        self.mfcc_list = mfcc_list
        
        self.mfcc_list = []
        self.transcripts = []
        for index, mfcc_file in enumerate(mfcc_list):
            self.mfcc_dir = data_path + "/test/" + mfcc_file + "/"

            mfcc_names = sorted(os.listdir(self.mfcc_dir))

            self.mfccs = []

            for i in range(0, len(mfcc_names)):
                mfcc = np.load(self.mfcc_dir + mfcc_names[i])
                self.mfccs.append(mfcc)
            #   Optionally do Cepstral Normalization of mfcc
            #   Load the corresponding transcript

            # if partition == "train":
            #     if shuffle == True:
            #         Pairs = list(zip(self.mfccs, self.transcripts))
            #         random.shuffle(Pairs)
            #         self.mfccs, self.transcripts = zip(*Pairs)

            # Each mfcc is of shape T1 x 20, T2 x 20, ...
            # Each transcript is of shape (T1+2) x 20, (T2+2) x 20

            # TODO: Concatenate all mfccs in self.X such that the final shape is T x 20 (Where T = T1 + T2 + ...) 
            self.mfccs = np.concatenate(self.mfccs, axis=0)
            self.length = len(self.mfccs)
            # TODO: Concatenate all transcripts in self.Y such that the final shape is (T,) meaning, each time step has one phoneme output
            # Hint: Use numpy to concatenate
            # Take some time to think about what we have done. self.mfcc is an array of the format (Frames x Features). Our goal is to recognize phonemes of each frame
            # From hw0, you will be knowing what context is. We can introduce context by padding zeros on top and bottom of self.mfcc
            if context != 0:
                zero_paddings = np.zeros((context, 20))
                up_paded = np.vstack((zero_paddings, self.mfccs))
                down_paded = np.vstack((up_paded, zero_paddings))
                self.mfccs = down_paded
                
            self.mfcc_list.append(self.mfccs)
        self.mfccs = np.stack(self.mfcc_list)

    def __len__(self):
        return self.length

    def __getitem__(self, ind):

        start_index = ind + self.offset - self.context
        
        ## Calculate ending timestep using offset and context (1 line)
        end_index = ind + self.offset + self.context + 1

        frames = self.mfccs[:, start_index:end_index, :]
        
        # TODO: Based on context and offset, return a frame at given index with context frames to the left, and right.
        # After slicing, you get an array of shape 2*context+1 x 15. But our MLP needs 1d data and not 2d.
        height = frames.shape[0]
        width = frames.shape[1]
        if self.normalization == True:
          frames = frames - frames.mean(axis=0, keepdims=True)
          # frames_variance = np.var(frames, axis=0)
          # frames = np.divide(frames, np.tile(frames_variance, (height, 1)))
        frames = torch.FloatTensor(frames) # Convert to tensors     
        
        return frames

In [ ]:
# class AudioTestDataset(torch.utils.data.Dataset):
#     # TODO: Create a test dataset class similar to the previous class but you dont have transcripts for this
#     # Imp: Read the mfccs in sorted order, do NOT shuffle the data here or in your dataloader.
#     def __init__(self, data_path, context, offset=0, limit=-1, Normalization=True, mfcc_list=["mfcc", "mfcc_320", "mfcc_480"]): # Feel free to add more arguments

#         self.context = context
#         self.offset = context
#         self.data_path = data_path
#         self.mfcc_dir = data_path + '/' + "test" + "/mfcc/"
#         self.normalization = Normalization
#         mfcc_names = sorted(os.listdir(self.mfcc_dir))
#         self.mfcc_list = mfcc_list
        
#         self.mfcc_list = []
#         for mfcc_file in mfcc_list:
            
#             self.mfccs= []
#             for i in range(0, len(mfcc_names)):
#             #   Load a single mfcc
#                 mfcc = np.load(self.mfcc_dir + mfcc_names[i])
#                 self.mfccs.append(mfcc)

#             # NOTE:
#             # Each mfcc is of shape T1 x 20, T2 x 20, ...
#             # Each transcript is of shape (T1+2) x 20, (T2+2) x 20 before removing [SOS] and [EOS]

#             self.mfccs = np.concatenate(self.mfccs, axis=0)
#             self.length = len(self.mfccs)

#             # Take some time to think about what we have done. self.mfcc is an array of the format (Frames x Features). Our goal is to recognize phonemes of each frame
#             # From hw0, you will be knowing what context is. We can introduce context by padding zeros on top and bottom of self.mfcc
#             if context != 0:
#                 zero_paddings = np.zeros((context, 20))
#                 up_paded = np.vstack((zero_paddings, self.mfccs))
#                 down_paded = np.vstack((up_paded, zero_paddings))
#                 self.mfccs = down_paded
            
#             self.mfcc_list.append(self.mfccs)
#         self.mfccs = np.stack(self.mfcc_list)

#     def __len__(self):
#         return self.length

#     def __getitem__(self, ind):

#         start_index = ind + self.offset - self.context
#         ## Calculate ending timestep using offset and context (1 line)
#         end_index = ind + self.offset + self.context + 1
#         frames = self.mfccs[:, start_index:end_index, :]
#         height = frames.shape[0]
#         width = frames.shape[1]
#         if self.normalization == True:
#           frames = frames - frames.mean(axis=0, keepdims=True)

#         frames = torch.FloatTensor(frames) # Convert to tensors 

#         return frames

In [ ]:
config = {
    'epochs': 30,
    'batch_size' : 512,
    'context' : 15,
    'learning_rate' : 0.05,
    'architecture' : 'medium-cutoff'
    # Add more as you need them - e.g dropout values, weight decay, scheduler parameters
}

In [ ]:
data_path = root
train_data = AudioDataset(data_path, context = config['context'], offset=0, partition= "train", limit=-1)
val_data = AudioDataset(data_path, context = config['context'], offset=0, partition= "dev", limit=-1, shuffle=False) 
test_data = AudioTestDataset(data_path, context = config['context'], offset=0, limit=-1) 

In [ ]:
train_loader = torch.utils.data.DataLoader(train_data, num_workers= 4,
                                           batch_size=config['batch_size'], pin_memory= True,
                                           shuffle= True)

val_loader = torch.utils.data.DataLoader(val_data, num_workers= 2,
                                         batch_size=config['batch_size'], pin_memory= True,
                                         shuffle= False)

test_loader = torch.utils.data.DataLoader(test_data, num_workers= 2, 
                                          batch_size=config['batch_size'], pin_memory= True, 
                                          shuffle= False)

print("Batch size: ", config['batch_size'])
print("Context: ", config['context'])
print("Input size: ", (2*config['context']+1)*20)

print("Train dataset samples = {}, batches = {}".format(train_data.__len__(), len(train_loader)))
print("Validation dataset samples = {}, batches = {}".format(val_data.__len__(), len(val_loader)))
print("Test dataset samples = {}, batches = {}".format(test_data.__len__(), len(test_loader)))

Batch size:  512
Context:  15
Input size:  620
Train dataset samples = 126430, batches = 247
Validation dataset samples = 17865, batches = 35
Test dataset samples = 17977, batches = 36


/usr/local/lib/python3.8/dist-packages/torch/utils/data/dataloader.py:554: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(


In [ ]:
for i, data in enumerate(train_loader):
    frames, onset = data
    onset = torch.squeeze(onset, 1)
    print(frames.shape, onset.shape)
    break

torch.Size([512, 3, 31, 20]) torch.Size([512])


In [ ]:
for i, data in enumerate(test_loader):
    frames = data
    print(frames[100,:,:,:])
    print(frames.shape)
    break

tensor([[[-1.3593e+01,  0.0000e+00,  0.0000e+00,  ...,  0.0000e+00,
           0.0000e+00,  0.0000e+00],
         [-1.3593e+01,  0.0000e+00,  0.0000e+00,  ...,  0.0000e+00,
           0.0000e+00,  0.0000e+00],
         [-1.3597e+01, -6.0009e-03, -5.9454e-03,  ..., -1.4052e-03,
          -9.4166e-04, -4.7228e-04],
         ...,
         [ 4.0855e+00,  1.5775e+01,  5.1100e+00,  ...,  1.8668e+00,
           2.7870e-01, -1.3439e+00],
         [ 4.2134e+00,  1.6996e+01,  4.0529e+00,  ...,  2.7411e+00,
          -6.2075e-01, -3.3246e+00],
         [ 4.7463e+00,  1.6372e+01,  4.6678e+00,  ...,  3.0651e+00,
          -4.2165e-01, -2.1737e+00]],

        [[ 2.9439e+00,  0.0000e+00,  0.0000e+00,  ...,  0.0000e+00,
           0.0000e+00,  0.0000e+00],
         [ 2.9439e+00,  0.0000e+00,  0.0000e+00,  ...,  0.0000e+00,
           0.0000e+00,  0.0000e+00],
         [ 2.9397e+00, -6.0009e-03, -5.9454e-03,  ..., -1.4052e-03,
          -9.4166e-04, -4.7228e-04],
         ...,
         [-2.3709e+00, -3

In [ ]:
# class Network(torch.nn.Module):

#     def __init__(self, context):

#         super(Network, self).__init__()

#         input_size = 21 #Why is this the case?
#         num_classes = 3
#         dropout_rate = 0.3
#         self.backbone = torch.nn.Sequential(
#             torch.nn.Conv2d(input_size, 10, kernel_size=(5, 3)),
#             torch.nn.BatchNorm1d(num_features=10),
#             torch.nn.MaxPool2d(kernel_size=(1, 3)),
#             torch.nn.Conv2d(10, 20, kernel_size=3), 
#             torch.nn.BatchNorm1d(num_features=20),
#             torch.nn.MaxPool2d(kernel_size=(1, 3)),
#             torch.nn.Flatten()
#         )
#         self.cls_layer = torch.nn.Linear(in_features=64, out_features=num_classes)          


#     def forward(self, x):

#         feats = self.backbone(x)
#         out = self.cls_layer(feats)
#         return out

In [ ]:
class Network(torch.nn.Module):

    def __init__(self, context):

        super(Network, self).__init__()

        input_size = 3 #Why is this the case?
        num_classes = 3
        dropout_rate = 0.3
        self.conv1 = torch.nn.Conv2d(input_size, 10, kernel_size=(5, 3))
        self.bn1 = torch.nn.BatchNorm2d(num_features=10)
        self.pool1 = torch.nn.MaxPool2d(kernel_size=(1, 3))
        self.conv2 = torch.nn.Conv2d(10, 20, kernel_size=3)
        self.bn2 = torch.nn.BatchNorm2d(num_features=20)
        self.pool2 = torch.nn.MaxPool2d(kernel_size=(1, 2))
        self.flatten = torch.nn.Flatten()
        
        # self.backbone = torch.nn.Sequential(
        #     torch.nn.Conv2d(input_size, 10, kernel_size=(5, 3)),
        #     torch.nn.BatchNorm1d(num_features=10),
        #     torch.nn.MaxPool2d(kernel_size=(1, 3)),
        #     torch.nn.Conv2d(10, 20, kernel_size=3), 
        #     torch.nn.BatchNorm1d(num_features=20),
        #     torch.nn.MaxPool2d(kernel_size=(1, 3)),
        #     torch.nn.Flatten()
        # )
        
        self.cls_layer = nn.Sequential(
            torch.nn.Linear(in_features=1000, out_features=256),
            torch.nn.Sigmoid(),
            torch.nn.Dropout(dropout_rate),
            torch.nn.Linear(in_features=256, out_features=num_classes)
        )          

    def forward(self, x):
        feats = self.conv1(x)
        feats = self.bn1(feats)
        feats = self.pool1(feats)
        feats = self.conv2(feats)
        feats = self.bn2(feats)
        feats = self.pool2(feats)
        feats = self.flatten(feats)
        
        out = self.cls_layer(feats)
        return out



In [ ]:
model = Network(config['context']).to(device)
frames,onsets = next(iter(train_loader))
summary(model, frames.to(device))

                         Kernel Shape       Output Shape    Params Mult-Adds
Layer                                                                       
0_conv1                 [3, 10, 5, 3]  [512, 10, 27, 18]     460.0    218.7k
1_bn1                            [10]  [512, 10, 27, 18]      20.0      10.0
2_pool1                             -   [512, 10, 27, 6]         -         -
3_conv2                [10, 20, 3, 3]   [512, 20, 25, 4]     1.82k    180.0k
4_bn2                            [20]   [512, 20, 25, 4]      40.0      20.0
5_pool2                             -   [512, 20, 25, 2]         -         -
6_flatten                           -        [512, 1000]         -         -
7_cls_layer.Linear_0      [1000, 256]         [512, 256]  256.256k    256.0k
8_cls_layer.Sigmoid_1               -         [512, 256]         -         -
9_cls_layer.Dropout_2               -         [512, 256]         -         -
10_cls_layer.Linear_3        [256, 3]           [512, 3]     771.0     768.0

/usr/local/lib/python3.8/dist-packages/torchsummaryX/torchsummaryX.py:101: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  df_sum = df.sum()


,Kernel Shape,Output Shape,Params,Mult-Adds
Layer,,,,
0_conv1,"[3, 10, 5, 3]","[512, 10, 27, 18]",460.0,218700.0
1_bn1,[10],"[512, 10, 27, 18]",20.0,10.0
2_pool1,-,"[512, 10, 27, 6]",NaN,NaN
3_conv2,"[10, 20, 3, 3]","[512, 20, 25, 4]",1820.0,180000.0
4_bn2,[20],"[512, 20, 25, 4]",40.0,20.0
5_pool2,-,"[512, 20, 25, 2]",NaN,NaN
6_flatten,-,"[512, 1000]",NaN,NaN
7_cls_layer.Linear_0,"[1000, 256]","[512, 256]",256256.0,256000.0
8_cls_layer.Sigmoid_1,-,"[512, 256]",NaN,NaN


In [ ]:
use_amp = True
criterion = torch.nn.CrossEntropyLoss() #Defining Loss function 
optimizer = torch.optim.AdamW(model.parameters(), lr=0.001, betas=(0.9, 0.999), eps=1e-08, weight_decay=0.01)
# optimizer = AdamP(model.parameters(), lr=config['learning_rate']) #Defining Optimizer
# scheduler = torch.optim.lr_scheduler.ExponentialLR(optimizer, gamma=0.9)
scheduler = torch.optim.lr_scheduler.ConstantLR(optimizer, factor=0.333, total_iters=5, last_epoch=- 1, verbose=False)
scaler = torch.cuda.amp.GradScaler(enabled=use_amp)

/usr/local/lib/python3.8/dist-packages/torch/cuda/amp/grad_scaler.py:118: UserWarning: torch.cuda.amp.GradScaler is enabled, but CUDA is not available.  Disabling.
  warnings.warn("torch.cuda.amp.GradScaler is enabled, but CUDA is not available.  Disabling.")


In [ ]:
torch.cuda.empty_cache()
gc.collect()

17

# Train

In [ ]:
with torch.no_grad():
    for i, data in enumerate(train_loader):
      
        #TODO: 
        # Follow the following steps, and 
        # Add some print statements here for sanity checking

        #1. What values are you returning from the collate function
        #2. Move the features and target to <DEVICE>
        #3. Print the shapes of each to get a fair understanding 
        #4. Pass the inputs to the model
            # Think of the following before you implement:
            # 4.1 What will be the input to your model?
            # 4.2 What would the model output?
            # 4.3 Print the shapes of the output to get a fair understanding 

        # Calculate loss: https://pytorch.org/docs/stable/generated/torch.nn.CTCLoss.html
        # Calculating the loss is not straightforward. Check the input format of each parameter
        x, y= data
        y = torch.squeeze(y, 1).long()
        x = x.to(device)
        y = y.to(device)
        predictions = model(x)
        print(y.shape)
        print(predictions.shape)

        loss = criterion(predictions, y)
        print(f"loss: {loss}")

        break # one iteration is enough

/usr/local/lib/python3.8/dist-packages/torch/utils/data/dataloader.py:554: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(


torch.Size([512])
torch.Size([512, 3])
loss: 1.2082265615463257


In [ ]:
def train(model, optimizer, criterion, dataloader, epoch):

    model.train()
    total_train_loss = 0.0 #Monitoring Loss
    total_train_acc = 0.0

    batch_bar = tqdm(total=len(dataloader), dynamic_ncols=True, leave=False, position=0, desc='Train', ncols=5) 
    
    for i , (mfccs, indentities) in enumerate(dataloader):
        indentities = torch.squeeze(indentities, 1).long()
        mfccs = mfccs.to(device)
        indentities = indentities.to(device)
        with torch.autocast(device_type='cuda', dtype=torch.float16, enabled=use_amp):
            ### Forward Propagation
            logits = model(mfccs)
            ### Loss Calculation
            loss = criterion(logits, indentities)

        train_loss = loss.item()

        train_acc = float(torch.sum(logits.argmax(axis=1) == indentities) / indentities.shape[0])
        
        total_train_loss += loss.item()

        total_train_acc += float(torch.sum(logits.argmax(axis=1) == indentities) / indentities.shape[0])

        batch_bar.set_postfix(
    acc="{:.04f}%".format(train_acc),
    loss="{:.04f}".format(train_loss),
    lr="{:.04f}".format(float(optimizer.param_groups[0]['lr'])))
        ### Initialize Gradients
        optimizer.zero_grad()

        ### Backward Propagation
        scaler.scale(loss).backward()

        scaler.step(optimizer)

        ### Gradient Descent
        scaler.update()

        batch_bar.update()
  
    batch_bar.close()
    total_train_loss /= len(dataloader)
    total_train_acc /= len(dataloader)
    return train_loss, train_acc*100, optimizer.param_groups[0]["lr"]

In [ ]:
def eval(model, dataloader, binary=True):

    model.eval() # set model in evaluation mode
    batch_bar = tqdm(total=len(dataloader), dynamic_ncols=True, leave=False, position=0, desc='Train', ncols=5) 
    phone_true_list = []
    phone_pred_list = []
    total_val_loss = 0.0
    total_val_acc = 0.0

    for i, data in enumerate(dataloader):

        frames, identities = data
        ### Move data to device (ideally GPU)
        identities = torch.squeeze(identities, 1).long()
        frames, identities = frames.to(device), identities.to(device) 

        with torch.inference_mode(): # makes sure that there are no gradients computed as we are not training the model now
            ### Forward Propagation
            logits = model(frames)
            loss = criterion(logits, identities)
        
        
        val_loss = loss.item()

        val_acc = float(torch.sum(logits.argmax(axis=1) == identities) / identities.shape[0])
        
        total_val_loss += loss.item()

        total_val_acc += float(torch.sum(logits.argmax(axis=1) == identities) / identities.shape[0])

        batch_bar.set_postfix(
    acc="{:.04f}%".format(val_acc),
    loss="{:.04f}".format(val_loss))
        ### Get Predictions
        predicted_phonemes = torch.argmax(logits, dim=1)
        ### Store Pred and True Labels
        phone_pred_list.extend(predicted_phonemes.cpu().tolist())
        phone_true_list.extend(identities.cpu().tolist())
        
        # Do you think we need loss.backward() and optimizer.step() here?
    
        total_val_loss /= len(dataloader)

        total_val_acc /= len(dataloader)
        del frames, identities, logits
        torch.cuda.empty_cache()
        batch_bar.update()

    batch_bar.close()
    total_val_loss /= len(dataloader)

    total_val_acc /= len(dataloader)
    ### Calculate Accuracy
    accuracy = sklearn.metrics.accuracy_score(phone_true_list, phone_pred_list)
    if not binary:
      f1 = None
      recall = None
      precision = None
    else:
      f1 = sklearn.metrics.f1_score(phone_true_list, phone_pred_list)
      recall = sklearn.metrics.recall_score(phone_true_list,  phone_pred_list)
      precision = sklearn.metrics.precision_score(phone_true_list,  phone_pred_list)
    return accuracy*100, total_val_loss, total_val_acc, f1, recall, precision

In [ ]:
torch.cuda.empty_cache()

best_acc = 0.0 ### Monitor best accuracy in your run
binary = False

for epoch in range(config['epochs']):
    print("\nEpoch {}/{}".format(epoch+1, config['epochs']))

    train_loss, train_acc, learning_rate = train(model, optimizer, criterion, train_loader, epoch)

    print("\nEpoch {}/{}: \nTrain Acc {:.04f}%\t Train Loss {:.04f}\t Learning Rate {:.04f}".format(
    epoch + 1,
    config['epochs'],
    train_acc,
    train_loss,
    learning_rate))


    accuracy, total_val_loss, total_val_acc, f1, recall, precision = eval(model, val_loader, binary=binary)

    print("Validai Acc: {:.4f}\tValidation Loss: {:.2f}%\tAccuracy Score: {:.2f}%".format(total_val_acc, total_val_loss, accuracy))

    if binary:
      print("F1 score: {:.4f}\tRecall: {:.2f}\tPrecision: {:.2f}".format(f1, recall, precision))
    ### Save checkpoint if accuracy is better than your current best
    if accuracy >= best_acc:

      ### Save checkpoint with information you want
      torch.save({'epoch': epoch,
              'model_state_dict': model.state_dict(),
              'optimizer_state_dict': optimizer.state_dict(),
              'loss': train_loss,
              'acc': accuracy}, 
        './model_checkpoint.pth')
      
      best_valacc = accuracy
      

    # Is your training time very high? Look into mixed precision training if your GPU (Tesla T4, V100, etc) can make use of it 
    # Refer - https://pytorch.org/docs/stable/notes/amp_examples.html



Epoch 1/30


Train:   0%|          | 0/247 [00:00<?, ?it/s]

/usr/local/lib/python3.8/dist-packages/torch/amp/autocast_mode.py:202: UserWarning: User provided device_type of 'cuda', but CUDA is not available. Disabling
  warnings.warn('User provided device_type of \'cuda\', but CUDA is not available. Disabling')



Epoch 1/30: 
Train Acc 82.4268%	 Train Loss 0.5341	 Learning Rate 0.0003


Train:   0%|          | 0/35 [00:00<?, ?it/s]

...

Train:   0%|          | 0/247 [00:00<?, ?it/s]

/usr/local/lib/python3.8/dist-packages/torch/utils/data/dataloader.py:554: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(
/usr/local/lib/python3.8/dist-packages/torch/amp/autocast_mode.py:202: UserWarning: User provided device_type of 'cuda', but CUDA is not available. Disabling
  warnings.warn('User provided device_type of \'cuda\', but CUDA is not available. Disabling')



Epoch 30/30: 
Train Acc 92.6778%	 Train Loss 0.2518	 Learning Rate 0.0003


Train:   0%|          | 0/35 [00:00<?, ?it/s]

Validai Acc: 0.0007	Validation Loss: 0.00%	Accuracy Score: 78.66%


In [ ]:
def test(model, test_loader):
  ### What you call for model to perform inference?
  model.eval()

  ### List to store predicted phonemes of test data
  test_predictions = []

  ### Which mode do you need to avoid gradients?
  with torch.no_grad():

      for i, frames in enumerate(tqdm(test_loader)):

          frames = frames.float().to(device)             
          output = model(frames)
          predicted_phonemes = torch.argmax(output, axis=1)
          test_predictions.extend(predicted_phonemes.cpu().tolist())
        
  return test_predictions

def test_labels_generate(path):
    transcript_names = sorted(os.listdir(path))
    transcripts = []
    for name in transcript_names:
      transcript = np.load(path + name)
      transcripts.append(transcript)
    transcripts_final = np.concatenate(transcripts, axis=0)
    return transcripts_final

In [ ]:
predictions = test(model, test_loader)
ground_truth = test_labels_generate(root + '/' + "test" + "/note_identity/")
test_accuracy = sklearn.metrics.accuracy_score(predictions, ground_truth)
f1 = sklearn.metrics.f1_score(ground_truth, predictions, average='weighted')
precision = sklearn.metrics.precision_score(ground_truth, predictions, average='weighted')
recall = sklearn.metrics.recall_score(ground_truth, predictions, average='weighted')
print("Test Accuracy: {:.2f}".format(test_accuracy))
print("F1 score: {:.2f}".format(f1))
print("Recall Score: {:.2f}".format(recall))
print("Precision Score: {:.2f}".format(precision))

  0%|          | 0/36 [00:00<?, ?it/s]

Test Accuracy: 0.72
F1 score: 0.70
Recall Score: 0.72
Precision Score: 0.70
